<a href="https://colab.research.google.com/github/chen-star/llm_model_trainings/blob/main/3_2_transformer_impl_transformer_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> ⭐ Transformer Block ⭐

This notebook builds and demonstrates a single Transformer Block using PyTorch.

*  **AttentionHead Class**: A custom `AttentionHead` module is implemented, which calculates scaled dot-product attention with `is_causal=True`.

*  **MLP Class**: A `MLP` (Multi-Layer Perceptron) module is created, incorporating two linear layers and a GELU activation function.

*  **TransformerBlock Class**: The `TransformerBlock` module combines the `AttentionHead` and `MLP` components. It includes Layer Normalization before each sub-layer (attention and MLP) and applies residual connections, following the standard Transformer architecture.

# ✈ Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # for 3D plotting

import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import override

# 🔢 Hyperparameters

In [2]:
batch_size = 3
embedding_dimension = 13
context_window_size = 8

# [1] 👓 Single Attention Head

In [3]:
class AttentionHead(nn.Module):
  def __init__(self, embedding_dimension):
    super().__init__()

    # define W_Q, W_K, W_V
    self.q_layer = nn.Linear(embedding_dimension, embedding_dimension, bias=False)
    self.k_layer = nn.Linear(embedding_dimension, embedding_dimension, bias=False)
    self.v_layer = nn.Linear(embedding_dimension, embedding_dimension, bias=False)

    # define W0
    self.w0_layer = nn.Linear(embedding_dimension, embedding_dimension, bias=False)


  @override
  def forward(self, X):
    # Q = XW_Q
    # K = XW_K
    # V = XW_V
    Q = self.q_layer(X)
    K = self.k_layer(X)
    V = self.v_layer(X)

    attention_score = F.scaled_dot_product_attention(Q, K, V, is_causal=True)
    return self.w0_layer(attention_score)

# [2] 🏃 Single MLP

In [4]:
class MLP(nn.Module):
  def __init__(self, embedding_dimension, expansion: int=4):
    super().__init__()

    # define W1, Gelu, W2
    self.w1_layer = nn.Linear(embedding_dimension, expansion * embedding_dimension) # 4x expansion
    self.gelu = nn.GELU()
    self.w2_layer = nn.Linear(expansion * embedding_dimension, embedding_dimension) # 4x contraction


  @override
  def forward(self, X):
    W1 = self.w1_layer(X)
    GELU = self.gelu(W1)
    W2 = self.w2_layer(GELU)

    return W2

# [3] 🔲 Single Transformer Block

In [5]:
class TransformerBlock(nn.Module):
  def __init__(self, embedding_dimension):
    super().__init__()

    # Attention
    self.layerNorm_attention = nn.LayerNorm(embedding_dimension)
    self.attention_head = AttentionHead(embedding_dimension)

    # MLP / FeedForward
    self.layerNorm_mlp = nn.LayerNorm(embedding_dimension)
    self.mlp = MLP(embedding_dimension)


  @override
  def forward(self, X):
    # --- Attention ---
    # X -> layerNorm -> attention_head
    #                                     +   = output
    #                                X
    X = X + self.attention_head(self.layerNorm_attention(X))

    # --- MLP ---
    # X -> layerNorm -> mlp
    #                         +   = output
    #                     X
    X = X + self.mlp(self.layerNorm_mlp(X))

    return X

In [6]:
transformer_block = TransformerBlock(embedding_dimension)
print(transformer_block)

TransformerBlock(
  (layerNorm_attention): LayerNorm((13,), eps=1e-05, elementwise_affine=True)
  (attention_head): AttentionHead(
    (q_layer): Linear(in_features=13, out_features=13, bias=False)
    (k_layer): Linear(in_features=13, out_features=13, bias=False)
    (v_layer): Linear(in_features=13, out_features=13, bias=False)
    (w0_layer): Linear(in_features=13, out_features=13, bias=False)
  )
  (layerNorm_mlp): LayerNorm((13,), eps=1e-05, elementwise_affine=True)
  (mlp): MLP(
    (w1_layer): Linear(in_features=13, out_features=52, bias=True)
    (gelu): GELU(approximate='none')
    (w2_layer): Linear(in_features=52, out_features=13, bias=True)
  )
)


In [7]:
# pass data once to test
random_X = torch.randn(batch_size, context_window_size, embedding_dimension)
output = transformer_block(random_X)
print(f"Output shape: \n{output.shape}\n")
print(f"Output:  \n{output}")

Output shape: 
torch.Size([3, 8, 13])

Output:  
tensor([[[-6.7200e-01, -1.0021e+00,  6.4806e-02,  6.9468e-01, -2.0332e+00,
          -1.7624e-01, -5.5038e-01, -1.0213e+00, -1.8001e+00,  4.8956e-01,
          -2.1546e-01,  1.0365e+00, -1.4063e+00],
         [-5.3165e-01,  1.1005e+00,  3.9205e-01,  6.1127e-01, -1.4813e+00,
           1.3676e+00,  2.0045e+00,  6.4204e-01, -1.9562e-03, -9.0771e-01,
           2.1294e-01,  1.7055e+00, -3.5207e+00],
         [-1.6778e-01, -1.1007e+00,  7.3168e-02, -2.0428e-02,  6.6337e-01,
           2.2267e+00, -1.2982e+00,  3.3505e-01,  2.4017e+00,  3.0655e-02,
          -9.4797e-01, -1.3617e+00,  4.6341e-01],
         [-4.3264e-01,  1.5660e+00,  4.2658e-01, -1.1551e+00, -2.0015e-01,
           7.7103e-02,  5.1251e-01, -1.8463e+00,  5.8794e-02,  4.1444e-01,
           2.3806e-01, -8.0845e-01, -2.3914e+00],
         [ 1.5072e+00,  1.0973e+00, -2.8834e-01,  4.2662e-02,  1.6436e-01,
           3.1724e-02,  2.8492e-01, -1.1419e+00, -2.8594e-01,  6.2059e-01,
 